In [1]:
import json
import pandas as pd

In [23]:
tree_path = '../auspice/flu_seasonal_h3n2_ha_6y_fra_tree.json'
output_csv = 'test_clades.csv'
with open(tree_path, 'r') as tree_json:
    tree = json.load(tree_json)

leaf_paths = {}
branch_muts = {}

def find_path(branch):

    node_path.append(branch['clade'])
    if branch['clade'] not in branch_muts.keys():
        branch_muts[str(branch['clade'])] = branch['muts']
    if 'children' in branch.keys():
        for child in branch['children']:
            find_path(child)
            node_path.remove(child['clade'])


    elif 'children' not in branch.keys():

        node_path_list = [str(node) for node in node_path]
        leaf_paths[branch['strain']] = [branch['attr']['clade_membership'], node_path_list]

node_path = []
find_path(tree)

df = pd.DataFrame(leaf_paths).T
df.reset_index(inplace=True)
df.columns = ['strain', 'clade_membership', 'path']
df['clade'] = 'unassigned'

#Find clades
max_path_length=df['path'].map(len).max()

current_clade = 0
assigned_clades = {}

for internal_branch in reversed(range(0,max_path_length)):
    exclude_assigned = df[df['clade']=='unassigned']
    sub_df = exclude_assigned[exclude_assigned['path'].map(len) > internal_branch]

    path_group= sub_df.groupby((sub_df.path.apply(lambda col: col[0:(internal_branch+1)])).map(tuple))
    for k, v in path_group:
        if len(v)>=100:
            current_clade+=1
            df.at[v.index, 'clade']=current_clade
            assigned_clades[current_clade] = {'clade_mrca':k[-1]}

        elif len(v)>=50:
            if len(branch_muts[str(k[-1])]) >= 1:
                current_clade+=1
                df.at[v.index, 'clade']=current_clade
                assigned_clades[current_clade] = {'clade_mrca':k[-1]}

        elif len(v)>=20:
            if len(branch_muts[str(k[-1])]) >= 4:
                current_clade+=1
                df.at[v.index, 'clade']=current_clade
                assigned_clades[current_clade] = {'clade_mrca':k[-1]}

df = df.set_index('strain')

#Find mutations that occur on most recent common ascestor of kk_clades
def find_defining_genotypes(branch):

    if 'children' in branch.keys():
        for child in branch['children']:
            for k,v in assigned_clades.items():
                if str(branch['clade']) == v['clade_mrca']:
                    assigned_clades[k]['aa_muts'] = branch['aa_muts']
                    assigned_clades[k]['nt_muts'] = branch['muts']
            find_defining_genotypes(child)

find_defining_genotypes(tree)
assigned_clades_df = pd.DataFrame(assigned_clades).T.reset_index().rename(columns={'index':'kk_clade'})
assigned_clades_df = assigned_clades_df[['kk_clade', 'clade_mrca', 'aa_muts', 'nt_muts']]
assigned_clades_df.to_csv(output_csv, index=False)

def add_clades(branch, clade_assignments):
    if 'children' in branch.keys():
        for child in branch['children']:
            add_clades(child, df)

    elif 'children' not in branch.keys():
        assigned_clade = df.loc[branch['strain']]['clade']
        branch['attr']['kk_clade'] = assigned_clade

add_clades(tree, df)

with open(tree_path, 'w') as outfile:
    json.dump(tree, outfile, indent=2, sort_keys = True)

In [21]:
df.groupby('clade').size()

clade
1              79
2              26
3             141
4             109
5              66
6              78
7              92
8              72
9              50
10             55
11             71
12            127
13            100
14             86
15             71
16             54
17            118
18             84
19             46
20             94
21             50
22            130
23             70
24             61
25             66
26            108
27            100
28            100
29             56
30             89
31             61
32             64
33             27
34             57
35             46
36             56
unassigned     18
dtype: int64